In [2]:
from utils.bech32m import convertbits, bech32_encode, Encoding
from utils.key import ECKey, ECPubKey, generate_bip340_key_pair
from utils.bip158 import gcs_match_any

from bip32 import BIP32
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException

import hashlib
def sha256(s):
    if type(s) != bytes:
        s = s.get_bytes()
    return hashlib.new('sha256', s).digest()

# Silent Payments

Using a new address for each Bitcoin transaction is a crucial aspect of maintaining privacy. This often requires a secure interaction between sender and receiver so that the receiver can hand out a fresh address, a batch of fresh address, or a way for the send to generate addresses, such as an xpub.

However, interaction is often infeasible and in many cases undesirable. To solve for this, various protocols have been proposed which use a static payment address and notifications, sent via the blockchain[footnote]. These protocols eliminate the need for interaction, but at the expense of increased costs for one-time payments and a noticeable footprint in the blockchain, potentially revealing metadata about the sender and receiver. Notification schemes also allow the receiver to link all payments from the same sender, compromising sender privacy.

This proposal aims to address the limitations of these current approaches by presenting a solution that eliminates the need for interaction, eliminates the need for notifications, and protects both sender and receiver privacy.

## Goals

We aim to present a transaction protocol which satisifies the following properties:

* No increase in the size or cost of transactions
* Resulting transactions blend in with other bitcoin transactions and can't be distinguished
* Transactions can't be linked to a silent payment address by an outside observer
* No sender-receiver interaction required
* No linking of multiple payments to the same sender
* Each silent payment goes to a unique address, avoiding accidental reuse
* Supports payment purpose labeling
* Uses existing seed phrase or descriptor methods for backup and recovery
* Separates scanning and spending responsibilities
* Compatible with other spending protocols, such as CoinJoin
* Light client/SPV wallet support
* Protocol is upgradeable


## What this workshop will attempt to cover

* Generating a silent payment address
* Scanning for silent payments
* Sync the wallet using BIP158 block filters
* BONUS:
  * Add label support
  * Scan for multiple outputs
  * Run on signet
  
## What this workshop won't cover

* Sending to silent payment addresses

# Elliptic Curve math review

Elliptic Curve math involves scalars and points.

* A scalar is a positive integer which is smaller than the group order, and is denoted by a lower case letter (eg `a`).
* A point lies on the curve and is denoted by an upper-case letter (eg `C`) or a pair of co-ordinates (eg `(x,y)`).

In Bitcoin, key pair generation and signing is performed over the secp256k1 curve. All scalars are modulo the group order `SECP256K1_ORDER`, which is a very large number

![test](images/ec_math0.jpg)

_An overview of all operations of scalars and points over elliptic curves._

## Simple case

Alice publishes a public key *A* as her silent payment address. Bob selects an input *B,b* and tweaks Alice's public key to create a destination public key *D* in the following way: 

* Let *D = HASH(b·A)·G + A* 

Bob constructs a transaction in the normal way with *B* as an input and *D* as the destination address. Alice detects this payment by computing:

* Let *D = HASH(a·B)·G + A* (Diffie-Hellman Key Exchange).

In [3]:
b, B = generate_bip340_key_pair()
a, A = generate_bip340_key_pair()

# Bob generates the output using his private key b and Alice's public key
# sha256(a * B)
t = sha256(b * A)
T = ECKey().set(t).get_pubkey()
D = T + A

# Alice checks if D is hers using her private key and Bob's public key
t_prime = sha256(a * B)
T_prime = ECKey().set(t_prime).get_pubkey()
D_prime = T_prime + A

assert D == D_prime

### Sending to more than one output

For additional privacy, Bob can derive multiple outputs for Alice in the following manner:

* Let *D<sub>0</sub> = HASH(b·A \|\| 0)·G + A*
    * This is defined as the *0-tweaked address*
* For additional outputs:
    * Let *D<sub>i</sub> = HASH(b·A \|\| n)·G + A*, where *n* starts from 1 and is incremented for each subsequent output

Alice detects this output the same as before by searching for *D<sub>0</sub> = HASH(a·B \|\| 0)·G + A*. Once she detects the *0-tweaked address*, she must:

* Check for *D<sub>1</sub> = HASH(B·a \|\| 1)·G + A*
    * If *D<sub>1</sub>* is not found, she  is done
* If *D<sub>1</sub>* is found, she continues to check for *D<sub>2</sub>* and so on until an output is not found

Since Alice will only perform these subsquent checks after a transaction with a *0-tweaked address* paying her is found, the increase to her overall scanning requirement is negligible.

## Preventing address reuse

If Bob were to use a different UTXO from the same public key *B* for a subseqent payment, he would end up deriving the same destination for Alice. To prevent this, Bob must include a hash of the outpoint in the following manner:

* Let *outpoint_hash = HASH(txid \|\| vout)* 
* Let *D<sub>0</sub> = HASH(outpoint_hash·b·A)·G + A*


In [4]:
# calculate the outpoint hash

txid = b'f4184fc596403b9d638783cf57adfe4c75c605f6356fbc91338530e9831e9e16'
vout = (0).to_bytes(4, 'little')

# sha256(txid || vout)
outpoint_hash = sha256(txid + vout)

t = sha256(outpoint_hash * b * A)
T = ECKey().set(t).get_pubkey()
D = T + A

# Alice scans..

t_prime = sha256(outpoint_hash * a * B)
T_prime = ECKey().set(t_prime).get_pubkey()
D_prime = T_prime + A

assert D == D_prime

### Using all inputs

In our simplified example we have been referring to Bob's transactions as having only one input *B*, but in reality a Bitcoin transaction can have many inputs. This complicates our protocol by forcing Bob to choose a particular input and requires Alice to check each input for each transaction. 

We can simplify things by requiring Bob to tweak Alice's public key *A* with the sum of his input private keys:

* Let *outpoints_hash = HASH(txid<sub>1</sub> \|\| vout<sub>1</sub> \|\| .. txid<sub>n</sub> \|\| vout<sub>n</sub>)*
* Let *inputs<sub>priv</sub> = i<sub>1</sub> + i<sub>2</sub> .. + i<sub>n</sub>*
* Let *D<sub>0</sub> = HASH(outpoints_hash·inputs<sub>priv</sub>·A)·G + A*

Alice now only needs to compute one tweak per transaction with the sum of the inputs, instead of computing a tweak per input.

In [5]:
# doesn't have to be a taproot UTXO
txid1 = b'f4184fc596403b9d638783cf57adfe4c75c605f6356fbc91338530e9831e9e16'
vout1 = (0).to_bytes(4, 'little')

b1 = ECKey().generate()
B1 = b1.get_pubkey()

# taproot UTXO
txid2 = b'f4184fc596403b9d638783cf57adfe4c75c605f6356fbc91338530e9831e9e16'
vout2 = (0).to_bytes(4, 'little')

b2, B2 = generate_bip340_key_pair()

outpoint_hash = sha256(txid1 + vout1 + txid2 + vout2)
t = sha256(outpoint_hash * (b1 + b2) * A)
T = ECKey().set(t).get_pubkey()
D = T + A

# alice
t_prime = sha256(outpoint_hash * a * (B1 + B2))
T_prime = ECKey().set(t_prime).get_pubkey()
D_prime = T_prime + A

assert D_prime == D

### Spend and Scan Key

Since Alice needs *a* to check for incoming payments, this requires her private key to be unencrypted in an online device. To mitigate this, Alice can instead publish an address of the form *(A<sub>spend</sub>, A<sub>scan</sub>)* where both are BIP340 public keys and Alice controls both private keys *a<sub>spend</sub>,a<sub>scan</sub>*.

This allows Alice to keep *a<sub>spend</sub>* in offline cold storage and perform the scanning with the public key *A<sub>spend</sub>* and private key *a<sub>scan</sub>*. 

Bob performs the tweak using both of Alice's public keys in the following manner:

* Let *D<sub>0</sub> = HASH(outpoints_hash·inputs<sub>priv</sub>·A<sub>scan</sub>)·G + A<sub>spend</sub>*

Alice detects this payment by calculating:

* *D<sub>0</sub> = HASH(outpoints_hash·inputs<sub>pub</sub>·a<sub>scan</sub>)·G + A<sub>spend</sub>*
* Spendable with *a<sub>spend</sub> + HASH(outpoints_hash·inputs<sub>pub</sub>·a<sub>scan</sub>) mod P* as the private key.


In [6]:
a_scan, A_scan = generate_bip340_key_pair()
a_spend, A_spend = generate_bip340_key_pair()

# doesn't have to be a taproot UTXO
txid1 = b'f4184fc596403b9d638783cf57adfe4c75c605f6356fbc91338530e9831e9e16'
vout1 = (0).to_bytes(4, 'little')

b1 = ECKey().generate()
B1 = b1.get_pubkey()

# taproot UTXO
txid2 = b'f4184fc596403b9d638783cf57adfe4c75c605f6356fbc91338530e9831e9e16'
vout2 = (0).to_bytes(4, 'little')

b2, B2 = generate_bip340_key_pair()

outpoint_hash = sha256(txid1 + vout1 + txid2 + vout2)
t = sha256(outpoint_hash * (b1 + b2) * A_scan)
T = ECKey().set(t).get_pubkey()
D = T + A_spend

# alice
t_prime = sha256(outpoint_hash * a_scan * (B1 + B2))
T_prime = ECKey().set(t_prime).get_pubkey()
D_prime = T_prime + A_spend

assert D_prime == D

### Labels

For a single silent payment address of the form *(A<sub>spend</sub>,A<sub>scan</sub>)*, Alice may wish to differintiate incoming payments by using labels. Naively, Alice could publish multiple silent payment addresses, but this would require her to scan for each silent address, which becomes prohibitively expense.

Instead, Alice can tweak the spend public key *A<sub>spend</sub>* with an integer *m* in the following way:

* Let *A<sub>m</sub> = A<sub>spend</sub> + m·G*
* Publish *(A<sub>0</sub>, A<sub>scan</sub>)*, *(A<sub>1</sub>, A<sub>scan</sub>)*

Bob performs the tweak same as before using one of the published *(A<sub>m</sub>,A<sub>scan</sub>)* pairs. Alice detects the labeled payment in the following manner:

* Let *D = HASH(outpoints_hash·inputs<sub>pub</sub>·a<sub>scan</sub>)·G + A<sub>spend</sub>*
* Compute *D<sub>m</sub> = D + m·G* for each *m*
* For each *D<sub>m</sub>* in {*D<sub>0</sub> .. D<sub>m</sub>*}, check if they are present in the transaction outputs


In [7]:
a_scan, A_scan = generate_bip340_key_pair()
a_spend, A_spend = generate_bip340_key_pair()

# create some labels
labels = {
    'twitter': A_spend + ECKey().set(1).get_pubkey(),
    'github': A_spend + ECKey().set(2).get_pubkey(),
    'project a': A_spend + ECKey().set(3).get_pubkey(),
    'project b': A_spend + ECKey().set(4).get_pubkey(),
}

# doesn't have to be a taproot UTXO
txid1 = b'f4184fc596403b9d638783cf57adfe4c75c605f6356fbc91338530e9831e9e16'
vout1 = (0).to_bytes(4, 'little')

b1 = ECKey().generate()
B1 = b1.get_pubkey()

# taproot UTXO
txid2 = b'f4184fc596403b9d638783cf57adfe4c75c605f6356fbc91338530e9831e9e16'
vout2 = (0).to_bytes(4, 'little')

b2, B2 = generate_bip340_key_pair()

# Bob finds Alice's silent payment address through github and makes a payment
A_2 = labels['github']

outpoint_hash = sha256(txid1 + vout1 + txid2 + vout2)
t = sha256(outpoint_hash * (b1 + b2) * A_scan)
T = ECKey().set(t).get_pubkey()
D = T + A_2

# alice
t_prime = sha256(outpoint_hash * a_scan * (B1 + B2))
T_prime = ECKey().set(t_prime).get_pubkey()

D_prime = None
for k,V in labels.items():
    if T_prime + V == D:
        print(f"Found payment to label {k}")
        D_prime = T_prime + V
        
assert D_prime == D

Found payment to label github


## Putting it all together

### Simple wallet: generating silent payment keys

* Use BIP32 hardened derivation for the silent payment keys
* Generate a silent payment address


In [8]:
class SPWallet:
    
    REASON = "1337"

    def __init__(self, seed):
        self.master = BIP32.from_seed(bytes.fromhex(seed))
        self.spend_path = f"m/{self.REASON}'/0'/0'"
        self.scan_path = f"m/{self.REASON}'/1'/0'"
        self.scan_privkey, self.scan_pubkey = self.convert_to_bip340_key_pair(
            self.get_scan_privkey()
        )
        self.spend_privkey, self.spend_pubkey = self.convert_to_bip340_key_pair(
            self.get_spend_privkey()
        )
        
    def convert_to_bip340_key_pair(self, seckey):
        d = ECKey().set(seckey)
        P = d.get_pubkey()
        if P.get_y() % 2 != 0:
            d.negate()
            P.negate()
        return d, P
        
    def get_scan_pubkey(self):
        return self.master.get_pubkey_from_path(self.scan_path)
    
    def get_scan_privkey(self):
        return self.master.get_privkey_from_path(self.scan_path)
    
    def get_spend_pubkey(self):
        return self.master.get_pubkey_from_path(self.spend_path)
    
    def get_spend_privkey(self):
        return self.master.get_privkey_from_path(self.spend_path)
    
    def get_silent_payment_address(self):
        pubkeys = self.scan_pubkey.get_bytes() + self.spend_pubkey.get_bytes()
        data = convertbits(pubkeys, 8, 5)
        return bech32_encode("sprt", data, Encoding.BECH32M)

### Scanning:  BIP158 block filters

* Get the tweak data per transaction (using `getsilentpaymentblockdata` rpc)
* Do the ECDH tweaks
* Check if any outputs exist in a block using BIP158 block filters

In [9]:
class SPScanner:
    PUBKEY_BYTES = 33
    TRUNC_HASH_BYTES = 8
    WITNESS_VERSION_1 = '5120'
    
    def __init__(self, spend_pubkey, scan_privkey, rpc_client, start_height):

        self.spend_pubkey = spend_pubkey
        self.scan_privkey = scan_privkey
        self.start_from_height = start_height
        self.last_block_scanned = self.start_from_height - 1
        self.rpc = rpc_client
        
    def sha256_secp256k1_ecdh(self, x32, y32):
        
        version = (y32[31] & 0x01) | 0x02
        sha = hashlib.sha256()
        sha.update(bytes([version]))
        sha.update(bytes(x32))
        
        return sha.digest()
        
    def refresh(self, rpc_client):
        self.rpc = rpc_client
        
    def scan(self, start=None):
    
        if start:
            self.start_from_height = start
            
        # get the chain height
        stop = self.rpc.getblockchaininfo()['blocks']
        current_block = self.start_from_height
        while current_block <= stop:
            res = self.get_silent_payment_block_data(current_block)
            if res['total_tx'] == 0:
                current_block += 1
                continue
            
            tweak_data = self.parse_silent_payment_block_data(res)
            outputs_to_check = self.compute_outputs(tweak_data)
            self.is_mine(outputs_to_check, res['block_hash'])
            current_block += 1
            
        self.start_from_height = current_block
        print("done scanning")
        
    def get_silent_payment_block_data(self, height):
            
        block_hash = self.rpc.getblockhash(height)
        data = self.rpc.getsilentpaymentblockdata(block_hash)
        data['block_hash'] = block_hash
        return data
        
    def parse_silent_payment_block_data(self, data):
        total_txs = data['total_tx']
        tx_data = data['data']
        txs = [
            tx_data[i:i + (self.PUBKEY_BYTES + self.TRUNC_HASH_BYTES)*2]
            for i in range(0, len(tx_data), (self.PUBKEY_BYTES + self.TRUNC_HASH_BYTES)*2)
        ]
        tweaks = [
            (tx[:self.PUBKEY_BYTES*2], tx[self.PUBKEY_BYTES*2:]) for tx in txs
        ]
        assert len(tweaks) == total_txs
        return tweaks
    
    def compute_outputs(self, txs):
        potential_outputs = []
        for (sum_pubkeys, trunc_hash) in txs:
            # compute the outpoint hash tweak
            I = ECPubKey().set(bytes.fromhex(sum_pubkeys))
            outpoint_hash = sha256(bytes.fromhex(trunc_hash))
            
            # ECDH
            ecdh = outpoint_hash * self.scan_privkey * I
            
            # SHA256(outpoint_hash * a * B)
            shared_secret = self.sha256_secp256k1_ecdh(
                ecdh.get_x().to_bytes(32, 'big'),
                ecdh.get_y().to_bytes(32, 'big'),
            )
            
            # shared_secret_hash = sha256(shared_secret)
            pubkey = ECKey().set(shared_secret).get_pubkey() + self.spend_pubkey
            
            # create a taproot scriptPubKey
            spk = self.WITNESS_VERSION_1 + pubkey.get_bytes().hex()
            potential_outputs.append(spk)
            
        return potential_outputs
    
    def get_compact_block_filter(self, block_hash):
        res = self.rpc.getblockfilter(block_hash)
        
        # cheating here, because this is actually a compactSize field
        N = int(res['filter'][:2], 16)
        block_filter = res['filter'][2:]
        return {'size': N, 'filter': block_filter}
    
    def is_mine(self, script_pub_keys, block_hash):
        
        gcs_data = self.get_compact_block_filter(block_hash)
        if gcs_match_any(block_hash, script_pub_keys, gcs_data):
            block_data = self.rpc.getblock(block_hash, 2)
            for tx in block_data['tx']:
                for vout in tx['vout']:
                    if vout['scriptPubKey']['hex'] in script_pub_keys:
                        print(f"{tx['txid']} is mine!")
        
def rpc_client():
    return AuthServiceProxy("http://%s:%s@127.0.0.1:18443"%("silent", "payments"))

wallet = SPWallet(sha256(b'stuff n fluff').hex())
wallet.get_silent_payment_address()

'sprt140xy8d8ys6t67tx8swea0dhhs0shjlzlrw6r6cdhteccessszd79hu2vw6xw8dnz9x54f4lfd92c2gflfrmzwa023l6ufu9y84hkg9cyxcejf'

In [10]:
scanner = SPScanner(wallet.spend_pubkey, wallet.scan_privkey, rpc_client(), 315)
scanner.scan()

f59fe48b899ee2448ff45761f268ef5ae2d461974d987cc12534b69f03c9f468 is mine!
9c064ece8a98bbdf4abbd30110a968940e617aca2fc0cb1d91e8aa3b3e70a596 is mine!
97f02878e92a87d96d1cdfea8105a93da39d3a9e2be88c68608e165e87baf9ad is mine!
de649cbee72051f57c768aeefe703af0ea91e197fb801651d8fccc49f60091d1 is mine!
46f5f22a83526709f9d93abbde1e4e941ccc6d442a96ee47493fc9da9e9a8ded is mine!
done scanning
